In [ ]:
import keras
keras.__version__

# Geração de texto com LSTM


## Implementando geração de texto LSTM em nível de caractere


Vamos colocar essas idéias em prática em uma implementação Keras. A primeira coisa que precisamos é de muitos dados de texto que possamos usar para aprender um
modelo de linguagem. Você pode usar qualquer arquivo de texto suficientemente grande ou conjunto de arquivos de texto - Wikipedia, o Senhor dos Anéis, etc.
Por exemplo, usaremos alguns dos escritos de Nietzsche, o filósofo alemão do final do século 19 (traduzidos para o inglês). O modelo de linguagem
vamos aprender que será, portanto, especificamente um modelo do estilo de escrita de Nietzsche e tópicos de escolha, ao invés de um modelo mais genérico do
Língua Inglesa.

## Preparando os dados

Vamos começar baixando o corpus e convertendo-o em letras minúsculas:

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 32s 53us/step
Corpus length: 600901


Em seguida, iremos extrair sequências parcialmente sobrepostas de comprimento `maxlen`, codificá-las one-hot e empacotá-las em uma matriz Numpy 3D `x` de
forma `(sequências, maxlen, caracteres_unicos)`. Simultaneamente, preparamos um array `y` contendo os alvos correspondentes: o one-hot
caracteres codificados que vêm logo após cada sequência extraída.

In [3]:
# Comprimento das sequências de caracteres extraídas
maxlen = 60

# Amostramos uma nova sequência a cada caractere `step`
step = 3

# Isso contém nossas sequências extraídas
sentences = []

# Isso mantém os alvos (os caracteres de acompanhamento)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Número de sequências:', len(sentences))

# Lista de caracteres únicos no corpus
chars = sorted(list(set(text)))
print('Caracters únicos:', len(chars))

# Dicionário mapeando caracteres únicos para seu índice em `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Em seguida, codifique one-hot os caracteres em matrizes binárias.
print('Vetorização...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
Unique characters: 59
Vectorization...


## Construindo a rede

Nossa rede é uma única camada `LSTM` seguida por um classificador` Dense` e softmax sobre todos os caracteres possíveis. Mas vamos notar que
as redes neurais recorrentes não são a única maneira de gerar a sequência de dados; Convnets 1D também provaram ser extremamente bem sucedidos nisso em
recentemente.

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Uma vez que nossos alvos são codificados em um ponto, usaremos `categorical_crossentropy` como a perda para treinar o modelo:

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Treinar o modelo de linguagem e fazer sua amostragem


Dado um modelo treinado e um snippet de texto original, geramos novo texto repetidamente:

* 1) Desenhar a partir do modelo uma distribuição de probabilidade sobre o próximo caractere de acordo com o texto disponível até agora
* 2) Reavaliando a distribuição a uma certa "temperatura"
* 3) Amostragem do próximo caractere aleatoriamente de acordo com a distribuição reponderada
* 4) Adicionar o novo caractere no final do texto disponível

Este é o código que usamos para reponderar a distribuição de probabilidade original que sai do modelo,
e extrair um índice de caracteres a partir dele (a "função de amostragem"):

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finalmente, este é o loop em que treinamos e geramos texto repetidamente. Começamos a gerar texto usando uma gama de temperaturas diferentes
depois de cada época. Isso nos permite ver como o texto gerado evolui conforme o modelo começa a convergir, bem como o impacto de
temperatura na estratégia de amostragem.

In [9]:
import random
import sys

#for epoch in range(1, 60):
for epoch in range(1, 2):
    print('epoch', epoch)
    # Ajustar o modelo para 1 época nos dados de treinamento disponíveis
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Selecione uma semente de texto aleatoriamente
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Gerando com semente: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperatura:', temperature)
        sys.stdout.write(generated_text)

        # Geramos 400 caracteres
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 162s 104ms/step - loss: 1.3281
--- Gerando com semente: "he was annoyed by the grandiose manner, the
mise en scene st"
------ temperatura: 0.2
he was annoyed by the grandiose manner, the
mise en scene still long and their sense of the subjugate to the spirit of the feeling of the subjugate of the subjugate of the subjugate of the spirits of the standard of the subjugate the more subjugate of the subjugate of the problem of the supposing still a self-concere the states of the sense of the subjugate the same their sense of the spirits of the soul and the highest for the same thing to the extrainone
------ temperatura: 0.5
he soul and the highest for the same thing to the extrainoness, the power of the new same men of their thing of his standard of the sense, the ture school-path--above them in the free spirits the sense of the preserved the life of a theological proportions of the german effect conditions of the opposite with the faith

Como você pode ver, uma temperatura baixa resulta em texto extremamente repetitivo e previsível, mas onde a estrutura local é altamente realista: em
Em particular, todas as palavras (sendo uma palavra um padrão local de caracteres) são palavras inglesas reais. Com temperaturas mais altas, o texto gerado
torna-se mais interessante, surpreendente e até criativo; às vezes pode inventar palavras completamente novas que soam um tanto plausíveis (como
"eterned" ou "troveration"). Com uma temperatura alta, a estrutura local começa a quebrar e a maioria das palavras parecem strings semi-aleatórias
de personagens. Sem dúvida, aqui 0,5 é a temperatura mais interessante para geração de texto nesta configuração específica. Sempre experimente
com múltiplas estratégias de amostragem! Um equilíbrio inteligente entre estrutura aprendida e aleatoriedade é o que torna a geração interessante.

Observe que ao treinar um modelo maior, mais longo, em mais dados, você pode obter amostras geradas que parecerão muito mais coerentes e
realista do que o nosso. Mas, claro, não espere gerar qualquer texto significativo, a não ser por acaso: tudo o que estamos fazendo é
dados de amostragem de um modelo estatístico de quais personagens vêm depois de quais personagens. A linguagem é um canal de comunicação, e há
uma distinção entre o que são as comunicações e a estrutura estatística das mensagens nas quais as comunicações são codificadas. Para
para comprovar essa distinção, aqui está um experimento de pensamento: e se a linguagem humana fizesse um trabalho melhor em compactar as comunicações, bem como
nossos computadores fazem com a maioria de nossas comunicações digitais? Então a linguagem não seria menos significativa, mas careceria de qualquer
estrutura estatística, tornando impossível aprender um modelo de linguagem como acabamos de aprender.


## Aprendizado

* Podemos gerar dados de sequência discreta treinando um modelo para prever os próximos tokens dados os tokens anteriores.
* No caso do texto, esse modelo é denominado "modelo de linguagem" e pode ser baseado em palavras ou caracteres.
* Amostrar o próximo token requer equilíbrio entre seguir o que o modelo julga provável e introduzir aleatoriedade.
* Uma maneira de lidar com isso é a noção de _softmax temperature_. Sempre experimente diferentes temperaturas para encontrar a "certa".